## Predicting a Successful New York Times Article Comment

IPython Process Book

An important part of your project is your iPython process book. Your process book details your steps in developing your solution, including how you collected the data, alternative solutions you tried, describing statistical methods you used, and the insights you got. Equally important to your final results is how you got there! Your process book is the place you describe and document the space of possibilities you explored at each step of your project. We strongly advise you to include many visualizations in your process book.Your process book should include the following topics. Depending on your project type the amount of discussion you devote to each of them will vary:


<strong>Overview and Motivation</strong>: Provide an overview of the project goals and the motivation for it. Consider that this will be read by people who did not see your project proposal.

<strong>Related Work</strong>: Anything that inspired you, such as a paper, a web site, or something we discussed in class.

<strong>Initial Questions</strong>: What questions are you trying to answer? How did these questions evolve over the course of the project? What new questions did you consider in the course of your analysis?

<strong>Data</strong>: Source, scraping method, cleanup, storage, etc.

<strong>Exploratory Data Analysis</strong>: What visualizations did you use to look at your data in different ways? What are the different statistical methods you considered? Justify the decisions you made, and show any major changes to your ideas. How did you reach these conclusions?

<strong>Final Analysis</strong>: What did you learn about the data? How did you answer the questions? How can you justify your answers?

<strong>Presentation</strong>: Present your final results in a compelling and engaging way using text, visualizations, images, and videos on your project web site.

# Overview and Motivation

# Related Work

# Initial Questions

# Data

#  Exploratory Analysis

# Final Analysis